In [1]:
%maven org.apache.spark:spark-core_2.11:2.4.0
%maven org.datavec:datavec-spark_2.11:1.0.0-beta_spark_2

In [2]:
import org.apache.spark.SparkConf;
import org.apache.spark.api.java.JavaRDD;
import org.apache.spark.api.java.JavaSparkContext;
import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
import org.datavec.api.transform.TransformProcess;
import org.datavec.api.transform.schema.Schema;
import org.datavec.api.writable.Writable;
import org.datavec.spark.transform.SparkTransformExecutor;
import org.datavec.spark.transform.misc.StringToWritablesFunction;
import org.datavec.spark.transform.misc.WritablesToStringFunction;

import java.util.Date;
import java.util.List;

In [3]:
int numLinesToSkip = 0;
String delimieter = ",";
String baseDir = "./";
String fileName = "reports.csv";
String inputPath = baseDir + fileName;
String timeStamp = String.valueOf(new Date().getTime());
String outputPath = baseDir + "reports_process_" + timeStamp;

In [4]:
Schema inputDataSchema = new Schema.Builder()
    .addColumnsString("datetime", "severity", "location", "county", "state")
    .addColumnsDouble("lat", "lon")
    .addColumnsString("comment")
    .addColumnCategorical("type", "TOR", "WIND", "HAIL")
    .build();

In [5]:
TransformProcess top = new TransformProcess.Builder(inputDataSchema)
    .removeColumns("datetime", "severity", "location", "county", "state", "comment")
    .categoricalToInteger("type")
    .build()

In [6]:
int numActions = top.getActionList().size();
for (int i=0; i<numActions; i++) {
    System.out.println("\n\n=====================");
    System.out.println("--- Schema after step " + i + " (" + top.getActionList().get(i) + ")--");
    System.out.println(top.getSchemaAfterStep(i));
}



--- Schema after step 0 (DataAction(RemoveColumnsTransform([datetime, severity, location, county, state, comment])))--
Schema():
idx   name        type           meta data
0     "lat"       Double         DoubleMetaData(name="lat",allowNaN=false,allowInfinite=false)
1     "lon"       Double         DoubleMetaData(name="lon",allowNaN=false,allowInfinite=false)
2     "type"      Categorical    CategoricalMetaData(name="type",stateNames=["WIND","TOR","HAIL"])



--- Schema after step 1 (DataAction(CategoricalToIntegerTransform(columnName=type, columnIdx=2, stateNames=[TOR, WIND, HAIL], statesMap={WIND=1, HAIL=2, TOR=0})))--
Schema():
idx   name        type           meta data
0     "lat"       Double         DoubleMetaData(name="lat",allowNaN=false,allowInfinite=false)
1     "lon"       Double         DoubleMetaData(name="lon",allowNaN=false,allowInfinite=false)
2     "type"      Integer        IntegerMetaData(name="type",minAllowed=0,maxAllowed=2)



In [7]:
SparkConf sparkConf = new SparkConf();
sparkConf.setMaster("local[*]");
sparkConf.setAppName("Storm Reports Record Reader Transform");

JavaSparkContext sc = new JavaSparkContext(sparkConf);

JavaRDD<String> lines = sc.textFile(inputPath);

[IJava-executor-0] WARN org.apache.spark.util.Utils - Your hostname, phoenix resolves to a loopback address: 127.0.1.1; using 10.33.9.66 instead (on interface eno1)
[IJava-executor-0] WARN org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address
[IJava-executor-0] INFO org.apache.spark.SparkContext - Running Spark version 2.4.0
[IJava-executor-0] WARN org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[IJava-executor-0] INFO org.apache.spark.SparkContext - Submitted application: Storm Reports Record Reader Transform
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing view acls to: bzr0014
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing modify acls to: bzr0014
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing view acls groups to: 
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing modify acls gr

[IJava-executor-0] INFO org.spark_project.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@79636a8b{/metrics/json,null,AVAILABLE,@Spark}
[IJava-executor-0] INFO org.apache.spark.storage.memory.MemoryStore - Block broadcast_0 stored as values in memory (estimated size 107.2 KB, free 2.8 GB)
[IJava-executor-0] INFO org.apache.spark.storage.memory.MemoryStore - Block broadcast_0_piece0 stored as bytes in memory (estimated size 20.4 KB, free 2.8 GB)
[dispatcher-event-loop-4] INFO org.apache.spark.storage.BlockManagerInfo - Added broadcast_0_piece0 in memory on 10.33.9.66:38947 (size: 20.4 KB, free: 2.8 GB)
[IJava-executor-0] INFO org.apache.spark.SparkContext - Created broadcast 0 from textFile at $JShell$51.java:25


In [8]:
JavaRDD<List<Writable>> stormReports = 
    lines.map(new StringToWritablesFunction(new CSVRecordReader()));

In [10]:
JavaRDD<List<Writable>> processed = 
    SparkTransformExecutor.execute(stormReports, top);

EvalException: class org.apache.hadoop.mapred.FileInputFormat tried to access method com.google.common.base.Stopwatch.<init>()V (org.apache.hadoop.mapred.FileInputFormat is in unnamed module of loader jdk.jshell.execution.DefaultLoaderDelegate$RemoteClassLoader @769e7ee8; com.google.common.base.Stopwatch is in unnamed module of loader 'app')